In [1]:
'''
    20181227 o2o大赛第1版，简单入门
'''

import os, sys, pickle

import numpy as np
import pandas as pd
import math

from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from  sklearn.preprocessing import MinMaxScaler

In [7]:
#加载数据开始
dfoff = pd.read_csv('data/ccf_offline_stage1_train.csv')
dfon = pd.read_csv('data/ccf_online_stage1_train.csv')

dftest = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')

#pd.set_option('display.max_columns', None)   #设置显示的最大列，参考地址：https://blog.csdn.net/yuanxiang01/article/details/79479938
#在jupyter中不需要的上面的解决方法，就可以显示全部的数据

dfoff.head(5)   #这里直接显示数据即可，显示效果还特别好看，这样话，比

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [6]:
#简单统计
print('有优惠卷，购买产品：%d' % dfoff[(dfoff['Date_received']!='null') & (dfoff['Date']!='null')].shape[0])
print('有优惠卷，未购产品：%d' % dfoff[(dfoff['Date_received']!='null') & (dfoff['Date']=='null')].shape[0])
print('无优惠卷，购买产品：%d' % dfoff[(dfoff['Date_received']=='null') & (dfoff['Date']!='null')].shape[0])
print('无优惠卷，未购产品：%d' % dfoff[(dfoff['Date_received']=='null') & (dfoff['Date']=='null')].shape[0])

有优惠卷，购买产品：75382
有优惠卷，未购产品：977900
无优惠卷，购买产品：701602
无优惠卷，未购产品：0


In [8]:
#特征提取
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [10]:
#折扣率：有3种情况：'nan或null'表示没有打折，[0,1]表示折扣率，x:y表示满x减y
print('Discount_rate类型：\n', dfoff['Discount_rate'].unique())

Discount_rate类型：
 ['null' '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30' '50:20'
 '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1' '200:10'
 '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50' '0.75'
 '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100' '150:5']


In [11]:
#处理方式:4个函数
'''
    getDiscountType()：转换打折类型
    convertRate():折扣率
    getDiscountMan():满多少
    getDiscountJian():减多少
'''


#转换打折类型，有3种情况：'nan或null'表示没有打折，[0,1]表示折扣率，x:y表示满x减y
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

#折扣率
def convertRate(row):
    '''Convert discount to rate'''
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

#满多少
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

#减多少
def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

#处理数据的函数
def processData(df):
    #转换折扣率
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)   #扩展1列
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)

    print(df['discount_rate'].unique())

    return df

In [12]:
dfoff = processData(dfoff)
dftest = processData(dftest)

dfoff.head(5)

[ 1.          0.86666667  0.95        0.9         0.83333333  0.8         0.5
  0.85        0.75        0.66666667  0.93333333  0.7         0.6
  0.96666667  0.98        0.99        0.975       0.33333333  0.2         0.4       ]
[ 0.83333333  0.9         0.96666667  0.8         0.95        0.75        0.98
  0.5         0.86666667  0.6         0.66666667  0.7         0.85
  0.33333333  0.94        0.93333333  0.975       0.99      ]


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1


In [13]:
#距离
print('Distance类型：', dfoff['Distance'].unique())

#将距离str转为int
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())   #unique()去掉重复的数据
dftest['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())   #unique()去掉重复的数据

Distance类型： ['0' '1' 'null' '2' '10' '4' '7' '9' '3' '5' '6' '8']
[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 0  1 -1  2 10  4  7  9  3  5  6  8]


In [14]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,discount_type,discount_rate,discount_man,discount_jian,distance
0,1439408,2632,null,null,0,null,20160217,null,1.000000,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,1,0.866667,150,20,1
2,1439408,2632,8591,20:1,0,20160217,null,1,0.950000,20,1,0
3,1439408,2632,1078,20:1,0,20160319,null,1,0.950000,20,1,0
4,1439408,2632,8591,20:1,0,20160613,null,1,0.950000,20,1,0


领券日期

In [17]:
date_received = dfoff['Date_received'].unique()
date_received = sorted(date_received[date_received!='null'])

date_buy = dfoff['Date'].unique()
date_buy = sorted(date_buy[date_buy!='null'])

print('优惠券收到的日期从：', date_received[0], '到', date_received[-1])
print('优惠券消费的日期从：', date_buy[0], '到', date_buy[-1])

优惠券收到的日期从： 20160101 到 20160615
优惠券使用的日期从： 20160101 到 20160630


In [ ]:
关于领券日期的特征：
    weekday:{null,1,2,3,4,5,6,7}
    weekday_type:{1,0} {1-周六，周日领取的优惠券，0-周一到周五领取的优惠券}
    weekday_1:{1 0 0 0 0 0 0}   #独热编码表示一天正常
    weekday_2:{0 1 0 0 0 0 0}   #独热编码表示一天正常
    weekday_3:{0 0 1 0 0 0 0}   #独热编码表示一天正常
    weekday_4:{0 0 0 1 0 0 0}   #独热编码表示一天正常
    weekday_5:{0 0 0 0 1 0 0}   #独热编码表示一天正常
    weekday_6:{0 0 0 0 0 1 0}   #独热编码表示一天正常
    weekday_7:{0 0 0 0 0 0 1}   #独热编码表示一天正常